<a href="https://colab.research.google.com/github/phrasenmaeher/custom-audio-classification-tf/blob/main/custaudio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports
We start with the usual packages

Code for section 4 of the post at
[TDS/Medium](https://towardsdatascience.com/custom-audio-classification-with-tensorflow-af8c16c38689)

In [ ]:
!pip install --upgrade kapre wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 133kB 14.1MB/s 
     |████████████████████████████████| 163kB 17.1MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for kapre: filename=kapre-0.3.4-cp37-none-any.whl size=20608 sha256=06c50835f8d330fd4214939c873a299bbc0e374fcc5c7f05c7700007eaa23b74
  Stored in directory: /root/.cache/pip/wheels/f8/9e/dc/cc6f4989c58eee2913abac4fab60723ff91aa2a06c6de4280f
Successfully built kapre
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=b0da439024e5981d5b1bad0881d8d9718a076658c5b4e4bade284b16b84d72cb
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtool

In [ ]:
import random, string, os, tqdm, pickle, glob, logging, sys, shutil, datetime, gc, logging, io, argparse, itertools

import matplotlib.pyplot as plt
import sklearn.metrics
import numpy as np
import wandb

Imports for tensorflow and the network's layers, using audio-specific implementations from the kapre package

In [ ]:
#------for the network-----------#
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import Callback
from wandb.keras import WandbCallback

#------for kapre-----------------#
import kapre
from kapre import STFT, Magnitude, MagnitudeToDecibel, Delta, Frame
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer, get_stft_magnitude_layer

# Helper functions
Let's implement some helper functions

In [ ]:
def set_all_seeds(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
def get_logger(filename, filemode):
  logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M',
                    filename=filename,
                    filemode=filemode)
  # define a Handler which writes INFO messages or higher to the sys.stderr
  console = logging.StreamHandler()
  console.setLevel(logging.INFO)
  # set a format which is simpler for console use
  formatter = logging.Formatter('%(message)s')
  # tell the handler to use this format
  console.setFormatter(formatter)
  # add the handler to the root logger
  logging.getLogger('').addHandler(console)

  logger1 = logging.getLogger("Standard")
  
  return logger1

In [ ]:
def get_file_lists(tfr_dir):

  #list all files with training data
  train_shards = glob.glob(tfr_dir+"*_train.tfrecords", recursive=False)
  sorted(train_shards) #for same same input order over all runs 

  #list all files with test data
  test_shards = glob.glob(tfr_dir+"*_test.tfrecords", recursive=False)
  sorted(test_shards) #for same same input order over all runs 

  #list all files with validation data
  valid_shards = glob.glob(tfr_dir+"*_valid.tfrecords", recursive=False)
  sorted(valid_shards) #for same same input order over all runs 

  logger1.info(f"Number of training, test, and validation shards: {len(train_shards), len(test_shards), len(valid_shards)}")

  return train_shards, test_shards, valid_shards

In [ ]:
def parse_tfr_elem(element):
  '''
   extract the features from a TFrecord element
  '''
  
  parse_dict = {
      'x': tf.io.FixedLenFeature([], tf.int64),
      'y':tf.io.FixedLenFeature([], tf.int64),
      'label':tf.io.FixedLenFeature([], tf.int64),
      'feature' : tf.io.FixedLenFeature([], tf.string)}

  example_message = tf.io.parse_single_example(element, parse_dict)

  x = example_message['x']
  y = example_message['y']
  feature = example_message['feature']
  label = example_message['label']

  feature = tf.io.parse_tensor(feature, out_type=tf.float32)
  feature = tf.reshape(feature, shape=[x,y])

  return (feature, label)

In [ ]:
def load_dataset(tfrecords, cache_path=None, batch_size=8, seed=1337):
  
  AUTOTUNE = tf.data.experimental.AUTOTUNE #automatic optimizer for caching, prefetching (later on)
  
  options = tf.data.Options()
  options.experimental_deterministic = False #speed up file processing by not waiting for files to be in order

  dataset = tf.data.TFRecordDataset(tfrecords, buffer_size=100000000)
  dataset.with_options(options)

  dataset = dataset.map(map_func=parse_tfr_elem, num_parallel_calls=AUTOTUNE)
  dataset = dataset.prefetch(AUTOTUNE)
  if cache_path is not None:
    dataset = dataset.cache(cache_path)
  else:
    dataset = dataset.cache() #hold cache in RAM

  dataset = dataset.shuffle(10, seed=seed, reshuffle_each_iteration=True)
  dataset = dataset.batch(batch_size=batch_size)
  dataset = dataset.repeat() #always repeat to run multiple epochs and multiple test/val runs

  return dataset

In [ ]:
def get_strategy(xla=0, fp16=0, no_cuda=0):
  '''
  Determines the strategy under which the network is trained.
  
  From https://github.com/huggingface/transformers/blob/8eb7f26d5d9ce42eb88be6f0150b22a41d76a93d/src/transformers/training_args_tf.py
  
  returns the strategy object
  
  '''
  logger1.info("TensorFlow: setting up strategy")

  if xla:
    tf.config.optimizer.set_jit(True)

  gpus = tf.config.list_physical_devices("GPU")
    # Set to float16 at first
  if fp16:
    policy = tf.keras.mixed_precision.experimental.Policy("mixed_float16")
    tf.keras.mixed_precision.experimental.set_policy(policy)

  if no_cuda:
    strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
  else:
    try:
      tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    except ValueError:
      tpu = None
  
    if tpu:
    # Set to bfloat16 in case of TPU
      if fp16:
        policy = tf.keras.mixed_precision.experimental.Policy("mixed_bfloat16")
        tf.keras.mixed_precision.experimental.set_policy(policy)
      tf.config.experimental_connect_to_cluster(tpu)
      tf.tpu.experimental.initialize_tpu_system(tpu)
    
      strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    elif len(gpus) == 0:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    elif len(gpus) == 1:
      strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    elif len(gpus) > 1:
      # If you only want to use a specific subset of GPUs use `CUDA_VISIBLE_DEVICES=0`
      strategy = tf.distribute.MirroredStrategy()
    else:
      raise ValueError("Cannot find the proper strategy! Please check your environment properties.")

  logger1.info(f"Using strategy: {strategy}")
  return strategy


# Callbacks

In [ ]:
class ClassificationReportCallback(Callback):
    __slots__=["x", "y", "file_writer", "class_names", "name","batch_size", "labels", "frequency"]
    def __init__(self, x, y, file_writer, class_names, name, batch_size, labels, frequency=5, use_wandb_tracking:bool=False):
      super(ClassificationReportCallback, self).__init__()
      self.x = x
      self.y = y
      self.class_names = class_names
      self.file_writer = file_writer
      self.batch_size = batch_size
      self.labels = labels
      self.name = name
      self.frequency = frequency #create report every k epochs
      self.wandb = use_wandb_tracking

    def on_epoch_end(self, epoch, logs=None):
      if (epoch % self.frequency)==0:
        self.generate_classification_report(epoch)
    
    def generate_classification_report(self, epoch):
      test_pred_raw = self.model.predict(self.x, batch_size=self.batch_size)
      test_pred = np.argmax(test_pred_raw, axis=1)

      classification_report_text = sklearn.metrics.classification_report(self.y, test_pred, labels=self.labels, target_names=self.class_names, output_dict=False)
      classification_report_text = "\n".join(classification_report_text.splitlines()[:7])

      with self.file_writer.as_default():
        tf.summary.text(self.name, classification_report_text, step=epoch)

      if self.wandb:
        classification_report_dict = sklearn.metrics.classification_report(self.y, test_pred, labels=self.labels, target_names=self.class_names, output_dict=True)
        wandb.log(data=classification_report_dict)
      
      gc.collect()

In [ ]:
#after https://www.tensorflow.org/tensorboard/image_summaries

class ConfusionMatrixCallback(Callback):
    __slots__=['x', 'y', 'class_names', 'image_name', 'file_writer', 'batch_size', 'frequency']
    def __init__(self, x, y, file_writer, class_names, image_name, batch_size, frequency=5):
        super(ConfusionMatrixCallback, self).__init__()
        self.x = x
        self.y = y
        self.class_names = class_names
        self.image_name = image_name
        self.batch_size=batch_size
        self.file_writer = file_writer
        self.frequency = frequency


    def on_epoch_end(self, epoch, logs=None):
        if (epoch%self.frequency) == 0:
          self.log_confusion_matrix(epoch)

    
    def log_confusion_matrix(self, epoch):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = self.model.predict(self.x, batch_size=self.batch_size)
        test_pred = np.argmax(test_pred_raw, axis=-1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(self.y, test_pred)
        # Log the confusion matrix as an image summary.
        figure = self.plot_confusion_matrix(cm, class_names=self.class_names)
        cm_image = self.plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with self.file_writer.as_default():
          tf.summary.image(self.image_name, cm_image, step=epoch)
        
        gc.collect()


    def plot_confusion_matrix(self, cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
          color = "white" if cm[i, j] > threshold else "black"
          plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(self, figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


In [ ]:
def get_callbacks(validation_dataset, steps_valid, train_monitor_x, train_monitor_y, valid_monitor_x, valid_monitor_y, args):

  logger1.info("\nSetting up callbacks.")
  

  tensorboard_dir = args['out_dir']+"tensorboard/"
  os.makedirs(tensorboard_dir, exist_ok=True)

  file_writer = tf.summary.create_file_writer(tensorboard_dir)

  #tensorboard callback
  logger1.info("\nLogging TensorBoard to {}.".format(tensorboard_dir))
  
  #tensorboard callback
  tb_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1, write_graph=True, write_images=True)

  #confusion matrix callback (applied after each epoch)
  cm_callback_train = ConfusionMatrixCallback(train_monitor_x, train_monitor_y, file_writer, ["male", "female"], 'Confusion Matrix Train', args['batch_size'], 3)
  cm_callback_valid = ConfusionMatrixCallback(valid_monitor_x, valid_monitor_y, file_writer, ["male", "female"], 'Confusion Matrix Valid', args['batch_size'], 3)

  #Classification report callback (applied after each epoch)
  classification_report_cb = ClassificationReportCallback(valid_monitor_x, valid_monitor_y, file_writer, ["male", "female"], "Classification Report Validation", args['batch_size'], [0,1], 3)

  #early stopping
  es_callback = keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=args['es_patience'], verbose=1, restore_best_weights=True)


  callbacks = [tb_callback, es_callback, cm_callback_train, cm_callback_valid, classification_report_cb]
  
  if args['use_wandb_tracking']:
    wandb_callback = WandbCallback(labels=["male", "female"], log_weights=True, monitor="val_sparse_categorical_accuracy", generator=validation_dataset, validation_steps=steps_valid, log_evaluation=True)
    callbacks.append(wandb_callback)

  logger1.info("\nCallbacks are all set up.")

  return callbacks

# The model

In [ ]:
def get_model(config):
  input_ = layers.Input(shape=(1323000, 1))

  mel_layer = get_melspectrogram_layer(n_mels=config['n_mels'], sample_rate=22050, win_length=config['win_length'], n_fft=config['n_fft'], return_decibel=True, output_data_format='channels_last')(input_)

  delta_layer = Delta()(mel_layer)
  concat = layers.Concatenate()([mel_layer, delta_layer])
  frame_layer = Frame(frame_length=config['frame_length'], hop_length=config['hop_length'])(concat)

  x = layers.Conv2D(config['num_conv1'], config['kernel_size'], padding="same")(frame_layer)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.Dropout(config['dropout1'])(x)
  x = layers.MaxPooling3D(pool_size=config['pool_size'])(x)

  x = layers.Conv2D(config['num_conv2'], config['kernel_size'], padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.Dropout(config['dropout2'])(x)
  x = layers.MaxPooling3D(pool_size=config['pool_size'])(x)

  x = layers.Conv2D(config['num_conv3'], config['kernel_size'], padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.Dropout(config['dropout3'])(x)
  x = layers.MaxPooling3D(pool_size=config['pool_size'])(x)

  x = layers.Conv2D(config['num_conv4'], config['kernel_size'], padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)
  x = layers.Dropout(config['dropout4'])(x)
  x = layers.GlobalMaxPooling3D()(x)

  x = layers.Dropout(config['dropout5'])(x)
  x = layers.Dense(config['num_dense'], activation="relu", name="pre_out_dense")(x)
  x = layers.Dense(2, activation="softmax")(x)

  return keras.Model(inputs=[input_], outputs=[x], name="KapreClassifier")

In [ ]:
def getHyperparameterDefaults(args):
  hyperparameter_defaults = {
     'num_conv1':24, #conv should be multiple of 8
     'num_conv2':32,
     'num_conv3':64,
     'num_conv4':128,
     'dropout1':0.1,
     'dropout2':0.1,
     'dropout3':0.1,
     'dropout4':0.1,
     'pool_size': (2,2,2),
     'kernel_size': (3,3),
     'win_length': 512,
     'n_fft':2048,
     'n_mels':60,
     'frame_length':41,
     'hop_length':64,
     'dropout5':0.5,
     'num_dense':512, #dense should be multiple of 8
     'epochs':args['epochs']} #writing args here to log it to W&B (just in case)
  
  return hyperparameter_defaults

# Setup for training

In [ ]:
def setup_model(strategy, model_config, args):
  #setting the training strategy
  with strategy.scope():
    model = get_model(model_config)
    model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=["sparse_categorical_accuracy"])

  #saving a model's visualization
  tf.keras.utils.plot_model(
    model,
    to_file=args["out_dir"]+"model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=150,
  )

  return model

In [ ]:
def save_model(model, args):
  
  model_directory = os.path.join(args['out_dir'], args['model_dir'])
  
  os.makedirs(model_directory, exist_ok=True)

  model.save(model_directory)


In [ ]:
def evaluate_model(model):
  
  logger1.info("\nStarting evaluation on test data.")
  
  test_loss, test_acc = model.evaluate(x=test_dataset, steps=steps_test)
  
  logger1.info("\nFinished evaluation. Loss: {:.4f}, Accuracy: {:.4f}.".format(test_loss, test_acc))

  return test_loss, test_acc

In [ ]:
def train(model, callbacks, args):

  logger1.info("Starting training.")
  
  model.fit(x=train_dataset, validation_data=validation_dataset, validation_steps=steps_valid, steps_per_epoch=steps_train, epochs=args['epochs'], callbacks=callbacks)

  logger1.info("Training finished.")

  return model  


In [ ]:
def setup(args):

  #seeding for reproducability
  set_all_seeds(args['seed'])

  #get model configuration
  model_config = getHyperparameterDefaults(args)

  #initialize W&B logging if requested
  if args['use_wandb_tracking']:
    wandb.tensorboard.patch(root_logdir=args["out_dir"])
    wandb.init(config=model_config, entity=args["wandb_entity"], project=args["wandb_project"], group=str(args["wandb_project"]), sync_tensorboard=True)
  
  #determine the distribution strategy
  distribution_strategy = get_strategy(xla=args['xla'], fp16=args['fp16'], no_cuda=args['no_cuda'])

  #setup model
  model = setup_model(strategy=distribution_strategy, model_config=model_config, args=args)

  #initializing the callbacks
  callbacks = get_callbacks(validation_dataset, steps_valid, train_monitor_x, train_monitor_y, valid_monitor_x, valid_monitor_y, args)
  
  return model, callbacks

In [ ]:
def main(args):
  
  #initial setup
  model, callbacks = setup(args=args)

  #summary
  model.summary(positions=[.33, .60, .67, 1.])

  #training
  model = train(model=model, callbacks=callbacks, args=args)

  #saving the model
  save_model(model, args)  

  #evaluate model:
  test_loss, test_acc = evaluate_model(model)

  if args['use_wandb_tracking']:
    wandb.log({"Test loss":test_loss, "Test accuracy": test_acc})
    wandb.finish()
  
  tf.keras.backend.clear_session() #clean up, free memory (not reliable though)
  del model
  gc.collect()

  logger1.info("\nScript finished.")

# Commandline arguments

In [ ]:
parser = argparse.ArgumentParser(description='')

parser.add_argument('--out_dir', dest='out_dir', default="/content/custaudio/", help="Directory where all the stuff is stored")
parser.add_argument('--tensorboard_dir', dest='tensorboard_dir', default="tensorboard", help="Subdir to store the logs at")
parser.add_argument('--tfr_dir', dest='tfr_dir', default="/content/drive/MyDrive/custaudio/tfr_dir/", help="Where the TFRecord files are stored")
parser.add_argument('--logfile_name', dest='logfile_name', default="logfile.log", help="Where the logfile is stored")
parser.add_argument('--model_dir', dest='model_dir', default="model", help="Name of the directory to store the model")

parser.add_argument('--epochs', dest='epochs', type=int, default=1000, help="Number of training epochs")
parser.add_argument('--seed', dest='seed', type=int, default=1337, help="Seed for initializing random number generators")
parser.add_argument('--per_device_batch_size', dest='per_device_batch_size', type=int, default=8, help="Batch size per device")
parser.add_argument('--xla', dest='xla', type=int, default=1, help="Use xla compiled cpu operations")
parser.add_argument('--fp16', dest='fp16', type=int, default=0, help="Use mixed precision training")
parser.add_argument('--no_cuda', dest='no_cuda', type=int, default=0, help="Run training only on CPU")
parser.add_argument('--es_patience', dest='es_patience', type=int, default=25, help="Number of epochs without change until ES terminates training and restores the best weights so far")

parser.add_argument('--wandb', dest='use_wandb_tracking', type=int, default=0, help="Use wandb logging")
parser.add_argument('--entity', dest='wandb_entity',  help="Entity to log the wandb stuff to")
parser.add_argument('--group', dest='wandb_group', type=str, help="Group for the wandb run")
parser.add_argument('--project', dest='wandb_project', default="speaker_classification", help="Project to log the files to when using wandb")

args, unknown = parser.parse_known_args()
args = args.__dict__

# Driver code

In [ ]:
if __name__ == "__main__":
  os.makedirs(args['out_dir'], exist_ok=True)
  logger1 = get_logger(filename=args['out_dir']+args['logfile_name'], filemode="a")

  #defining the following variables here makes them available globally
  num_devices = len(tf.config.list_physical_devices('GPU')) 
  multiplier = num_devices if num_devices != 0 else 1
  batch_size = args['per_device_batch_size']*multiplier
  args['batch_size'] = batch_size
  logger1.info(f"Batch size is {batch_size}")
  
  #first load the shards, then create a dataset object from them
  train_shards, test_shards, valid_shards = get_file_lists(tfr_dir=args['tfr_dir'])
  train_dataset = load_dataset(tfrecords=train_shards, cache_path=None, batch_size=batch_size, seed=args['seed'])
  validation_dataset = load_dataset(tfrecords=valid_shards, cache_path=None, batch_size=batch_size, seed=args['seed'])
  test_dataset = load_dataset(tfrecords=test_shards, cache_path=None, batch_size=batch_size, seed=args['seed'])
  
  steps_train = 50//batch_size  
  steps_test = 50//batch_size 
  steps_valid = 50//batch_size

  logger1.info("Loading some batches to visualize learning")
  train_monitor_x = np.load(args['tfr_dir']+"train_x_monitor.npy") #for gradient visualization, cm plotting
  train_monitor_y = np.load(args['tfr_dir']+"train_y_monitor.npy")
  valid_monitor_x = np.load(args['tfr_dir']+"valid_x_monitor.npy") #for gradient visualization, cm plotting
  valid_monitor_y = np.load(args['tfr_dir']+"valid_y_monitor.npy")

  main(args)

Batch size is 8
Number of training, test, and validation shards: (1, 1, 1)
Loading some batches to visualize learning
TensorFlow: setting up strategy
Using strategy: <tensorflow.python.distribute.one_device_strategy.OneDeviceStrategy object at 0x7ff8105fba50>

Setting up callbacks.

Logging TensorBoard to /content/custaudio/tensorboard/.

Callbacks are all set up.
Starting training.


Model: "KapreClassifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape              Param  Connected to                     
input_1 (InputLayer)            [(None, 1323000, 1)]      0                                       
__________________________________________________________________________________________________
melspectrogram (Sequential)     (None, 10332, 60, 1)      0      input_1[0][0]                    
__________________________________________________________________________________________________
delta (Delta)                   (None, 10332, 60, 1)      0      melspectrogram[0][0]             
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 10332, 60, 2)      0      melspectrogram[0][0]             
                                                                 delta[0][0]        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 8/1000
6/6 [==============================] - 1s 147ms/step - loss: 1.1569 - sparse_categorical_accuracy: 0.5443 - val_loss: 0.6801 - val_sparse_categorical_accuracy: 0.7083
Epoch 9/1000
6/6 [==============================] - 1s 149ms/step - loss: 0.5235 - sparse_categorical_accuracy: 0.7904 - val_loss: 0.6948 - val_sparse_categorical_accuracy: 0.4792
Epoch 10/1000
6/6 [==============================] - 1s 134ms/step - loss: 1.0891 - sparse_categorical_accuracy: 0.5658 - val_loss: 0.7144 - val_sparse_categorical_accuracy: 0.4375
Epoch 11/1000
6/6 [==============================] - 1s 142ms/step - loss: 0.7812 - sparse_categorical_accuracy: 0.7069 - val_loss: 0.7203 - val_sparse_categorical_accuracy: 0.4167
Epoch 12/1000
6/6 [==============================] - 1s 138ms/step - loss: 0.9517 - sparse_categorical_accuracy: 0.4504 - val_loss: 0.6902 - val_sparse_categorical_accuracy: 0.5208
Epoch 13/1000
6/6 [==============================] - 1s 137ms/step - loss: 0.8291 - sparse_catego

Training finished.


Epoch 00028: early stopping
INFO:tensorflow:Assets written to: /content/custaudio/model/assets


Assets written to: /content/custaudio/model/assets

Starting evaluation on test data.


6/6 [==============================] - 4s 297ms/step - loss: 0.6901 - sparse_categorical_accuracy: 0.5625



Finished evaluation. Loss: 0.6901, Accuracy: 0.5625.

Script finished.
